In [20]:
import torch
from transformers import Blip2VisionConfig, Blip2VisionModel, AutoModelForImageClassification
import copy, os, sys
sys.path.append('../')
from minigpt4.models.eva_vit import VisionTransformer, create_eva_vit_g

/export/home/.cache/minigpt4


In [21]:
mini_ve =  create_eva_vit_g(precision='fp32')

incompatible_keys: 
 _IncompatibleKeys(missing_keys=[], unexpected_keys=['norm.weight', 'norm.bias', 'head.weight', 'head.bias', 'blocks.39.norm1.weight', 'blocks.39.norm1.bias', 'blocks.39.attn.q_bias', 'blocks.39.attn.v_bias', 'blocks.39.attn.qkv.weight', 'blocks.39.attn.proj.weight', 'blocks.39.attn.proj.bias', 'blocks.39.norm2.weight', 'blocks.39.norm2.bias', 'blocks.39.mlp.fc1.weight', 'blocks.39.mlp.fc1.bias', 'blocks.39.mlp.fc2.weight', 'blocks.39.mlp.fc2.bias'])


In [22]:
mini_ve

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1408, out_features=4224, bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1408, out_features=1408, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1408, out_features=6144, bias=True)
        (act): GELU(approximate=none)
        (fc2): Linear(in_features=6144, out_features=1408, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (a

In [29]:
mini_ve.blocks[0].attn.state_dict().keys()

odict_keys(['q_bias', 'v_bias', 'qkv.weight', 'proj.weight', 'proj.bias'])

In [271]:
i = 0
for x in mini_ve.modules():
    print(f'{i} {x}')
    i+=1

0 VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1408, out_features=4224, bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1408, out_features=1408, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1408, out_features=6144, bias=True)
        (act): GELU(approximate=none)
        (fc2): Linear(in_features=6144, out_features=1408, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      

<generator object Module.modules at 0x7fc0bc0aa580>

PeftModel(
  (base_model): LoraModel(
    (model): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (blocks): ModuleList(
        (0): Block(
          (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(
              in_features=1408, out_features=4224, bias=False
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=1408, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=16, out_features=4224, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
            )
            (attn_drop): Dropout(p=

In [35]:
## Test for qkv

In [234]:
dim = 5
qkv = torch.nn.Linear(dim, dim*3).eval()
lq = torch.nn.Linear(dim, dim).eval()
lk = torch.nn.Linear(dim, dim).eval()
lv = torch.nn.Linear(dim, dim).eval()

In [258]:
lq.weight = torch.nn.Parameter(qkv.weight[0:dim,:])
lk.weight = torch.nn.Parameter(qkv.weight[dim:2*dim,:])
lv.weight = torch.nn.Parameter(qkv.weight[2*dim:3*dim,:])

lq.bias = torch.nn.Parameter(qkv.bias[0:dim])
lk.bias = torch.nn.Parameter(qkv.bias[dim:2*dim])
lv.bias = torch.nn.Parameter(qkv.bias[2*dim:3*dim])

In [250]:
lq.weight.data = qkv.weight.data[0:dim,:]
lk.weight.data = qkv.weight.data[dim:2*dim,:]
lv.weight.data = qkv.weight.data[2*dim:3*dim,:]

lq.bias.data = qkv.bias.data[0:dim]
lk.bias.data = qkv.bias.data[dim:2*dim]
lv.bias.data = qkv.bias.data[2*dim:3*dim]

In [251]:
input_test = torch.randn([1,3,dim])

In [259]:
qkv_out = qkv(input_test)
lq_out = lq(input_test)
lk_out = lk(input_test)
lv_out = lv(input_test)

In [260]:
qkv_out.shape

torch.Size([1, 3, 15])

In [261]:
fqkv_out = torch.cat([lq_out,lk_out,lv_out], dim=2)

In [262]:
fqkv_out

tensor([[[-7.6996e-01, -6.1207e-02,  2.9218e-01,  9.5623e-01,  9.5634e-01,
           6.0143e-01,  5.0327e-01, -8.3091e-01,  1.3538e-02,  5.8970e-01,
           7.5116e-01,  5.2686e-01, -1.4199e+00,  7.1408e-01, -1.2708e+00],
         [-1.0409e+00,  3.5706e-01,  7.7207e-01, -8.1972e-03,  4.5232e-01,
           1.1576e+00,  1.5320e+00, -1.3131e-01,  5.9755e-01,  6.8769e-01,
           3.9372e-01,  1.1992e-03, -1.5298e+00,  1.0866e+00, -1.4098e+00],
         [-6.7180e-01, -6.0535e-01,  2.3557e-01,  7.7827e-01,  2.1809e-02,
          -3.3447e-01, -2.1217e-01,  3.9364e-01,  2.0960e-02,  2.5075e-01,
          -9.0930e-01,  7.5032e-01,  4.8156e-01,  2.4848e-01,  4.3870e-01]]],
       grad_fn=<CatBackward0>)

In [263]:
qkv_out

tensor([[[-7.6996e-01, -6.1207e-02,  2.9218e-01,  9.5623e-01,  9.5634e-01,
           6.0143e-01,  5.0327e-01, -8.3091e-01,  1.3538e-02,  5.8970e-01,
           7.5116e-01,  5.2686e-01, -1.4199e+00,  7.1408e-01, -1.2708e+00],
         [-1.0409e+00,  3.5706e-01,  7.7207e-01, -8.1972e-03,  4.5232e-01,
           1.1576e+00,  1.5320e+00, -1.3131e-01,  5.9755e-01,  6.8769e-01,
           3.9372e-01,  1.1992e-03, -1.5298e+00,  1.0866e+00, -1.4098e+00],
         [-6.7180e-01, -6.0535e-01,  2.3557e-01,  7.7827e-01,  2.1809e-02,
          -3.3447e-01, -2.1216e-01,  3.9364e-01,  2.0960e-02,  2.5075e-01,
          -9.0930e-01,  7.5032e-01,  4.8156e-01,  2.4848e-01,  4.3870e-01]]],
       grad_fn=<ViewBackward0>)

In [264]:
torch.max(fqkv_out - qkv_out)

tensor(1.1921e-07, grad_fn=<MaxBackward1>)

In [224]:
qkv.bias.shape

torch.Size([15])

In [39]:
lq.weight.shape

torch.Size([100, 100])

tensor([[[True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True],
         [True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True],
         [True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True]]])

In [21]:
from minigpt4.models.eva_vit_sparse_qkv import create_eva_vit_g_sparse_qkv

In [22]:
mini_ve =  create_eva_vit_g(precision='fp32')
s_mini_ve = create_eva_vit_g_sparse_qkv(precision='fp32')

incompatible_keys: 
 _IncompatibleKeys(missing_keys=[], unexpected_keys=['norm.weight', 'norm.bias', 'head.weight', 'head.bias', 'blocks.39.norm1.weight', 'blocks.39.norm1.bias', 'blocks.39.attn.q_bias', 'blocks.39.attn.v_bias', 'blocks.39.attn.qkv.weight', 'blocks.39.attn.proj.weight', 'blocks.39.attn.proj.bias', 'blocks.39.norm2.weight', 'blocks.39.norm2.bias', 'blocks.39.mlp.fc1.weight', 'blocks.39.mlp.fc1.bias', 'blocks.39.mlp.fc2.weight', 'blocks.39.mlp.fc2.bias'])
incompatible_keys: 
 _IncompatibleKeys(missing_keys=['blocks.0.attn.query.weight', 'blocks.0.attn.key.weight', 'blocks.0.attn.value.weight', 'blocks.1.attn.query.weight', 'blocks.1.attn.key.weight', 'blocks.1.attn.value.weight', 'blocks.2.attn.query.weight', 'blocks.2.attn.key.weight', 'blocks.2.attn.value.weight', 'blocks.3.attn.query.weight', 'blocks.3.attn.key.weight', 'blocks.3.attn.value.weight', 'blocks.4.attn.query.weight', 'blocks.4.attn.key.weight', 'blocks.4.attn.value.weight', 'blocks.5.attn.query.weight', 'b

In [23]:
mini_ve.eval()
s_mini_ve.eval()
input_image = torch.randn([1,3,224,224])
out_ns = mini_ve(input_image)
out_s = s_mini_ve(input_image)

In [24]:
torch.max(out_ns - out_s)

tensor(0., grad_fn=<MaxBackward1>)

In [25]:
False in (out_ns == out_s)

False

In [26]:
out_ns

tensor([[[ 0.6130,  0.3576,  0.4530,  ...,  0.0764, -2.6244,  1.3055],
         [ 0.5288,  1.2078,  0.2100,  ...,  0.9368, -0.4550,  0.4996],
         [ 0.3802,  1.0288,  0.6383,  ...,  0.6912, -0.3559, -0.1243],
         ...,
         [ 0.6755,  0.5917,  0.6956,  ...,  1.9820, -0.6884,  0.6076],
         [ 1.0455,  1.5453,  0.8666,  ...,  0.1916, -0.4121,  0.2025],
         [ 0.9976,  0.6633,  0.8597,  ...,  1.1618, -0.0878,  1.3839]],

        [[ 0.6146,  0.2851,  0.4449,  ...,  0.1012, -2.6816,  1.4402],
         [ 0.6663,  0.6814,  0.5441,  ...,  1.0736, -0.1627,  1.3004],
         [-0.1694,  0.8392,  0.5206,  ...,  0.9756, -0.2034,  0.5643],
         ...,
         [ 0.7379,  1.3083,  0.3780,  ...,  1.1864,  0.3446, -0.5274],
         [ 0.4539,  1.5651,  0.8978,  ...,  1.4355,  0.4208,  0.7676],
         [ 1.0197,  0.7783,  0.9767,  ...,  1.0601, -0.0200,  1.6157]],

        [[ 0.7601,  0.2384,  0.4656,  ...,  0.1436, -2.8049,  1.5256],
         [ 0.4584,  0.8854,  0.0455,  ...,  0

In [27]:
out_s

tensor([[[ 0.6130,  0.3576,  0.4530,  ...,  0.0764, -2.6244,  1.3055],
         [ 0.5288,  1.2078,  0.2100,  ...,  0.9368, -0.4550,  0.4996],
         [ 0.3802,  1.0288,  0.6383,  ...,  0.6912, -0.3559, -0.1243],
         ...,
         [ 0.6755,  0.5917,  0.6956,  ...,  1.9820, -0.6884,  0.6076],
         [ 1.0455,  1.5453,  0.8666,  ...,  0.1916, -0.4121,  0.2025],
         [ 0.9976,  0.6633,  0.8597,  ...,  1.1618, -0.0878,  1.3839]],

        [[ 0.6146,  0.2851,  0.4449,  ...,  0.1012, -2.6816,  1.4402],
         [ 0.6663,  0.6814,  0.5441,  ...,  1.0736, -0.1627,  1.3004],
         [-0.1694,  0.8392,  0.5206,  ...,  0.9756, -0.2034,  0.5643],
         ...,
         [ 0.7379,  1.3083,  0.3780,  ...,  1.1864,  0.3446, -0.5274],
         [ 0.4539,  1.5651,  0.8978,  ...,  1.4355,  0.4208,  0.7676],
         [ 1.0197,  0.7783,  0.9767,  ...,  1.0601, -0.0200,  1.6157]],

        [[ 0.7601,  0.2384,  0.4656,  ...,  0.1436, -2.8049,  1.5256],
         [ 0.4584,  0.8854,  0.0455,  ...,  0

In [28]:
from peft import LoraConfig, get_peft_config, get_peft_model
s_mini_ve.config = None
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=['query','key'],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
lora_model = get_peft_model(s_mini_ve, config)

In [ ]:
lora_model

In [37]:
lora_model.base_model.blocks[0].attn.query.lora_B.default.weight.shape

torch.Size([1408, 16])

In [31]:
mini_ve.eval()
lora_model.eval()
input_image = torch.randn([1,3,224,224])
out_ns = mini_ve(input_image)
out_s = s_mini_ve(input_image)

In [32]:
torch.max(out_ns - out_s)

tensor(0., grad_fn=<MaxBackward1>)

In [33]:
False in (out_ns == out_s)

False

In [38]:
out_ns.shape

torch.Size([1, 257, 1408])

In [39]:
257*1408

361856

In [34]:
mini_ve

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1408, out_features=4224, bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1408, out_features=1408, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1408, out_features=6144, bias=True)
        (act): GELU(approximate=none)
        (fc2): Linear(in_features=6144, out_features=1408, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (a

In [35]:
vision_config_hv = {'hidden_dropout_prob':0.0}

In [36]:
from transformers import ViTModel

In [11]:
hvit = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

In [ ]:
vision_config = Blip2VisionConfig()

In [6]:
blip_ve = Blip2VisionModel(vision_config)

In [9]:
or_bve = copy.deepcopy(blip_ve)

In [25]:
%%bash
export http_proxy=http://oversea-squid4.sgp.txyun:11080 https_proxy=http://oversea-squid4.sgp.txyun:11080 no_proxy=localhost,127.0.0.1,localaddress,localdomain.com,internal,corp.kuaishou.com,test.gifshow.com,staging.kuaishou.com

In [28]:
automodel = AutoModelForImageClassification.from_pretrained(
    pretrained_model_name_or_path = "google/vit-base-patch16-224-in21k",  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /hetu_group/wangyan/envs/env_minigpt4/lib/python3.9/site-packages/transformers/utils/hub.py:417  │
│ in cached_file                                                                                   │
│                                                                                                  │
│    414 │   user_agent = http_user_agent(user_agent)                                              │
│    415 │   try:                                                                                  │
│    416 │   │   # Load from URL or cache if already cached                                        │
│ ❱  417 │   │   resolved_file = hf_hub_download(                                                  │
│    418 │   │   │   path_or_repo_id,                                                              │
│    419 │   │   │   filename,                                                                     │
│    420 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ /hetu_group/wangyan/envs/env_minigpt4/lib/python3.9/site-packages/huggingface_hub/utils/_validat │
│ ors.py:118 in _inner_fn                                                                          │
│                                                                                                  │
│   115 │   │   if check_use_auth_token:                                                           │
│   116 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   117 │   │                                                                                      │
│ ❱ 118 │   │   return fn(*args, **kwargs)                                                         │
│   119 │                                                                                          │
│   120 │   return _inner_fn  # type: ignore                                                       │
│   121                                                                                            │
│                                                                                                  │
│ /hetu_group/wangyan/envs/env_minigpt4/lib/python3.9/site-packages/huggingface_hub/file_download. │
│ py:1291 in hf_hub_download                                                                       │
│                                                                                                  │
│   1288 │   │   │   │   " and downloads online, set 'local_files_only' to False."                 │
│   1289 │   │   │   )                                                                             │
│   1290 │   │   else:                                                                             │
│ ❱ 1291 │   │   │   raise LocalEntryNotFoundError(                                                │
│   1292 │   │   │   │   "Connection error, and we cannot find the requested files in"             │
│   1293 │   │   │   │   " the disk cache. Please try again or make sure your Internet"            │
│   1294 │   │   │   │   " connection is on."                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
LocalEntryNotFoundError: Connection error, and we cannot find the requested files in the disk cache. Please try 
again or make sure your Internet connection is on.

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 automodel = AutoModelFo

In [26]:
vision_config

Blip2VisionConfig {
  "attention_dropout": 0.0,
  "dropout": 0.0,
  "hidden_act": "gelu",
  "hidden_size": 1408,
  "image_size": 224,
  "initializer_factor": 1.0,
  "initializer_range": 1e-10,
  "intermediate_size": 6144,
  "layer_norm_eps": 1e-05,
  "model_type": "blip_2_vision_model",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 39,
  "patch_size": 14,
  "projection_dim": 512,
  "qkv_bias": true,
  "transformers_version": "4.31.0.dev0"
}